In [ ]:
"""Backend supported: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle"""
import deepxde as dde
import numpy as np
import tensorflow as tf

def func(x):
    """
    x: array_like, N x D_in
    y: array_like, N x D_out
    """
    return x * np.sin(5 * x)


geom = dde.geometry.Interval(-1, 1)
num_train = 16
num_test = 100
data = dde.data.Function(geom, func, num_train, num_test)

activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn([1] + [20] * 3 + [1], activation, initializer)

model = dde.Model(data, net)
model._compile_tensorflow(lr=0.001, loss_fn=any,decay=any,optimizer=Pylance)
losshistory, train_state = model.train(iterations=10000)

dde.saveplot(losshistory, train_state, issave=True, isplot=True)

In [1]:
import os

# Set the backend to TensorFlow
os.environ['DDE_BACKEND'] = 'tensorflow'

import deepxde as dde

# Your DeepXDE code here


Using backend: tensorflow
Other supported backends: tensorflow.compat.v1, pytorch, jax, paddle.
paddle supports more examples now and is recommended.


ModuleNotFoundError: No module named 'tensorflow_probability'

In [ ]:
"""Backend supported: tensorflow.compat.v1, paddle"""
import deepxde as dde
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# Import tf if using backend tensorflow.compat.v1
from deepxde.backend import tf
# Import paddle if using backend paddle
# import paddle
from scipy.special import gamma


alpha0 = 1.8
alpha = dde.Variable(1.5)


# Backend tensorflow.compat.v1
def fpde(x, y, int_mat):
    r"""\int_theta D_theta^alpha u(x)"""
    if isinstance(int_mat, (list, tuple)) and len(int_mat) == 3:
        int_mat = tf.SparseTensor(*int_mat)
        lhs = tf.sparse_tensor_dense_matmul(int_mat, y)
    else:
        lhs = tf.matmul(int_mat, y)
    lhs = lhs[:, 0]
    lhs *= -tf.exp(tf.lgamma((1 - alpha) / 2) + tf.lgamma((2 + alpha) / 2)) / (
        2 * np.pi ** 1.5
    )
    x = x[: tf.size(lhs)]
    rhs = (
        2 ** alpha0
        * gamma(2 + alpha0 / 2)
        * gamma(1 + alpha0 / 2)
        * (1 - (1 + alpha0 / 2) * tf.reduce_sum(x ** 2, axis=1))
    )
    return lhs - rhs
# Backend paddle
# def fpde(x, y, int_mat):
#     r"""\int_theta D_theta^alpha u(x)"""
#     if isinstance(int_mat, (list, tuple)) and len(int_mat) == 3:
#         indices, values, shape = int_mat
#         int_mat = paddle.sparse.sparse_coo_tensor(
#             [[p[0] for p in indices], [p[1] for p in indices]],
#             values,
#             shape,
#             stop_gradient=False
#         )
#         lhs = paddle.sparse.matmul(int_mat, y)
#     else:
#         lhs = paddle.mm(int_mat, y)
#     lhs = lhs[:, 0]
#     lhs *= -paddle.exp(paddle.lgamma((1 - alpha) / 2) + paddle.lgamma((2 + alpha) / 2)) / (
#         2 * np.pi ** 1.5
#     )
#     x = x[: paddle.numel(lhs)]
#     rhs = (
#         2 ** alpha0
#         * gamma(2 + alpha0 / 2)
#         * gamma(1 + alpha0 / 2)
#         * (1 - (1 + alpha0 / 2) * paddle.sum(x ** 2, axis=1))
#     )
#     return lhs - rhs


def func(x):
    return (1 - np.linalg.norm(x, axis=1, keepdims=True) ** 2) ** (1 + alpha0 / 2)


geom = dde.geometry.Disk([0, 0], 1)

observe_x = geom.random_points(30)
observe_y = dde.icbc.PointSetBC(observe_x, func(observe_x))

data = dde.data.FPDE(
    geom,
    fpde,
    alpha,
    observe_y,
    [8, 100],
    num_domain=64,
    anchors=observe_x,
    solution=func,
)

net = dde.nn.FNN([2] + [20] * 4 + [1], "tanh", "Glorot normal")
# Backend tensorflow.compat.v1
net.apply_output_transform(
    lambda x, y: (1 - tf.reduce_sum(x ** 2, axis=1, keepdims=True)) * y
)
# Backend paddle
# net.apply_output_transform(
#     lambda x, y: (1 - paddle.sum(x ** 2, axis=1, keepdim=True)) * y
# )

model = dde.Model(data, net)
model.compile("adam", lr=1e-3, loss_weights=[1, 100], external_trainable_variables=[alpha])
variable = dde.callbacks.VariableValue(alpha, period=1000)
losshistory, train_state = model.train(iterations=10000, callbacks=[variable])
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create a Sequential model
model = Sequential()

# Add layers to the model
model.add(Dense(units=64, activation='relu', input_shape=(784,)))
model.add(Dense(units=10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


In [ ]:
import deepxde as dde
import numpy as np


def func(x):
    return np.sin(5 * x)

# Geometry: the domain of the function
geom = dde.geometry.Interval(-1, 1)

# Generate training data based on the function
num_train = 100
num_test = 100
data = dde.data.Function(geom, func, num_train, num_test)

# Neural network: 1 input layer (dimension of x), 2 hidden layers of 20 neurons each, 1 output layer
layer_size = [1] + [20]*2 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.maps.FNN(layer_size, activation, initializer)

# Model
model = dde.Model(data, net)
model.compile("adam", lr=0.001)
losshistory, train_state = model.train(epochs=10000)
